<a href="https://colab.research.google.com/github/DhayananthB/HaSy_Recognition/blob/main/Hasy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
from torchvision import transforms
import torch.optim as optim

In [17]:
# !pip install deeplake
import deeplake


# Dataset Description
- HASY (Handwritten Symbol Dataset)
- 168,233 instances of 369 classes
- 3X32x32  images

### Reading data

In [18]:
ds = deeplake.load("hub://activeloop/hasy-train")

Opening dataset in read-only mode as you don't have write permissions.


|

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/hasy-train



\

hub://activeloop/hasy-train loaded successfully.



train sample size

In [19]:
len(ds)

151241

In [20]:
# ds.visualize()

In [21]:
dt = deeplake.load("hub://activeloop/hasy-test")

Opening dataset in read-only mode as you don't have write permissions.


-

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/hasy-test



/

hub://activeloop/hasy-test loaded successfully.



test sample size

In [22]:
len(dt)

16992

### transformation

In [23]:
transform_dict = {
    'images': transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))  # Normalize with mean=0.5, std=0.5
    ])
}

def custom_transform(sample):
    for key, transform in transform_dict.items():
        if key in sample:
            sample[key] = transform(sample[key])
    return sample

### Data Loading

In [24]:
train_dataloader = ds.pytorch(num_workers=0, batch_size=256, shuffle=True,decode_method={
        'images': 'pil'
    },transform=custom_transform)

In [25]:
test_dataloader = dt.pytorch(num_workers=0, batch_size=256, shuffle=True,decode_method={
        'images': 'pil'
    },transform=custom_transform)

In [26]:
dict(next(iter(test_dataloader))).keys()

dict_keys(['images', 'latex', 'index'])

## Neural Network

In [27]:
class HASYNet(nn.Module):
    def __init__(self):
        super(HASYNet, self).__init__()
        # Convolutional Layers
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)  # (3, 32, 32) -> (32, 32, 32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)  # (32, 32, 32) -> (64, 32, 32)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)  # (64, 32, 32) -> (128, 32, 32)

        # Pooling Layer
        self.pool = nn.MaxPool2d(2, 2)  # Reduces size by half each time

        # Calculate the size of the input to the fully connected layer
        # After 3 pooling layers (assuming input size is 32x32):
        # Size will be reduced to 32 / 2^3 = 4 (height and width)
        self.fc1_input_size = 128 * 4 * 4  # (128 channels) * (4 height) * (4 width)

        # Fully Connected Layers
        self.fc1 = nn.Linear(self.fc1_input_size, 512)
        self.fc2 = nn.Linear(512, 369)  # 369 classes for output

        # Activation
        self.relu = nn.ReLU()

        # Dropout for regularization
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        # Forward pass through convolutional layers with ReLU and MaxPool
        x = self.pool(self.relu(self.conv1(x)))  # (batch_size, 3, 32, 32) -> (batch_size, 32, 16, 16)
        x = self.pool(self.relu(self.conv2(x)))  # (batch_size, 32, 16, 16) -> (batch_size, 64, 8, 8)
        x = self.pool(self.relu(self.conv3(x)))  # (batch_size, 64, 8, 8) -> (batch_size, 128, 4, 4)

        # Flatten the tensor
        x = x.view(-1, self.fc1_input_size)  # (batch_size, 128*4*4)

        # Fully Connected layers with ReLU and Dropout
        x = self.relu(self.fc1(x))  # (batch_size, 512)
        x = self.dropout(x)  # Apply Dropout
        x = self.fc2(x)  # (batch_size, 369) -> Output layer

        return x

In [28]:
device = 'cpu'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [29]:
model = HASYNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [32]:
model = HASYNet()  # Assuming HASYNet is defined elsewhere
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Number of epochs
num_epochs = 3

# Function to evaluate loss and accuracy
def evaluate_loss_and_accuracy(loader, model, criterion):
    total_loss = 0.0
    correct = 0
    total = 0
    model.eval()  # Set model to evaluation mode
    with torch.no_grad():
        for data in loader:
            inputs = data['images']
            labels = data['latex']

            inputs.to(device)

            # Ensure labels are in the correct format
            labels = labels.squeeze()  # Convert from shape [128, 1] to [128]

            labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    average_loss = total_loss / len(loader)
    accuracy = 100 * correct / total
    return average_loss, accuracy

# Training and validation loop
for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0

    for i, data in enumerate(train_dataloader, 0):
        inputs = data['images']
        labels = data['latex']

        inputs.to(device)

        labels = labels.squeeze()
         # Convert from shape [128, 1] to [128]

        labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:  # Print every 100 mini-batches
            print(f"[Epoch {epoch + 1}, Batch {i + 1}] Training loss: {running_loss / 100:.3f}")
            running_loss = 0.0

    # Calculate training loss and accuracy
    train_loss, train_accuracy = evaluate_loss_and_accuracy(train_dataloader, model, criterion)
    print(f'Epoch {epoch + 1} Training Loss: {train_loss:.4f}, Accuracy: {train_accuracy:.2f}%')

    # Calculate validation loss and accuracy
    val_loss, val_accuracy = evaluate_loss_and_accuracy(test_dataloader, model, criterion)
    print(f'Epoch {epoch + 1} Validation Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.2f}%')

print('Finished Training')

[Epoch 1, Batch 100] Training loss: 3.909
[Epoch 1, Batch 200] Training loss: 1.813
[Epoch 1, Batch 300] Training loss: 1.328
[Epoch 1, Batch 400] Training loss: 1.135
[Epoch 1, Batch 500] Training loss: 1.051
Epoch 1 Training Loss: 0.6842, Accuracy: 79.88%
Epoch 1 Validation Loss: 0.6649, Accuracy: 79.31%
[Epoch 2, Batch 100] Training loss: 0.896
[Epoch 2, Batch 200] Training loss: 0.886
[Epoch 2, Batch 300] Training loss: 0.825
[Epoch 2, Batch 400] Training loss: 0.851
[Epoch 2, Batch 500] Training loss: 0.798
Epoch 2 Training Loss: 0.5569, Accuracy: 82.82%
Epoch 2 Validation Loss: 0.5710, Accuracy: 81.47%
[Epoch 3, Batch 100] Training loss: 0.743
[Epoch 3, Batch 200] Training loss: 0.733
[Epoch 3, Batch 300] Training loss: 0.716
[Epoch 3, Batch 400] Training loss: 0.727
[Epoch 3, Batch 500] Training loss: 0.716
Epoch 3 Training Loss: 0.4940, Accuracy: 84.17%
Epoch 3 Validation Loss: 0.5254, Accuracy: 82.32%
Finished Training


In [34]:
torch.save(model.state_dict(), "hasy.pth")

In [ ]:
# for epoch in range(1):
#     running_loss = 0.0
#     for i, data in enumerate(train_dataloader, 0):
#         # Extracting the inputs and labels from the dictionary
#         inputs = data['images']
#         labels = data['latex']
#         labels = labels.squeeze()
#         optimizer.zero_grad()

#         outputs = model(inputs)

#         # print(outputs.shape)
#         # print(labels.shape)

#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()

#         running_loss += loss.item()
#         if i % 100 == 99:  # Print every 100 mini-batches
#             print(f"[{epoch + 1}, {i + 1}] loss: {running_loss / 100:.3f}")
#             running_loss = 0.0

# print('Finished Training')
